### [ 일변량 데이터 기반 회귀 모델 ]
- 주제 : 학습 시간에 따른 수능 점수 예측 서비스
- 데이터 : study_score_easy.csv
- 구성 : 피쳐(hour) + 타겟(score)
- 학습 : 지도학습 + 회귀
- 구현 : 인공신경망

[1] 모듈 로딩 및 데이터 준비 <hr>

In [42]:
# [1-1] 모듈 로딩
import pandas as pd             # 데이터 분석 및 처리용 모듈
import torch                    # 텐서 및 수치, 기본 함수용 모듈
import torch.nn as nn           # 인공신경망 관련 모듈
import torch.nn.functional as F # 인공신경망 함수(AF, LF, MF) 관련 모듈

from torchinfo import *

# [1-2] 데이터 준비
dataDF = pd.read_csv("../DATA/study_score_easy.csv")
dataDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   hours   1000 non-null   float64
 1   score   1000 non-null   float64
dtypes: float64(2)
memory usage: 15.8 KB


[2] ANN 모델 설계 <HR>

In [43]:
# -------------------------------------------------------------------------------
#               입력수           퍼셉트론수/출력수           AF(활성화함수)
# -------------------------------------------------------------------------------
# 입력층           1                     1                
# 은닉층1          1                    10                     ReLU
# 은닉층2         10                    15                     ReLU
# 출력층          15                     1                      -     회귀
# -------------------------------------------------------------------------------
# ★ Pytorch에는 입력층 클래스 존재 X -> 입력 텐서를 입력층으로 간주
# -------------------------------------------------------------------------------
# 클래스 이름 : ScoreModel
# 부모 클래스 : nn.Module
# 오버라이딩  : __init__()       : 층 구성 요소 인스턴스 생성
#             forward(self, x) : 순전파 진행 메서드
#                                x -> 입력층으로 간주!
# -------------------------------------------------------------------------------
class ScoreModel(nn.Module):
    # 층 구성 인스턴스 생성 메서드
    def __init__(self):
        super().__init__()
        self.hd1_layer = nn.Linear(1, 10)
        self.hd2_layer = nn.Linear(10, 15)
        self.out_layer = nn.Linear(15, 1)

    # 순전파 진행 메서드
    def forward(self, x):
        # 입력층 -> 은닉층 : 1 -> 10
        out = F.relu(self.hd1_layer(x)) # x: 입력층

        # 은닉층1 -> 은닉층2 : 10 -> 15
        out = F.relu(self.hd2_layer(out))

        # 은닉층2 -> 출력층 : 15 -> 1
        # 회귀모델로 AF 없음
        out = self.out_layer(out)

        return out

In [ ]:
# 모델 인스턴스 생성
model = ScoreModel()
summary(model, input_size=(1, 1))   # 1개의 피쳐, 값은 1개(열과 행)
                                    # param 수 : (입력 * 출력) + 편향

Layer (type:depth-idx)                   Output Shape              Param #
ScoreModel                               [1, 1]                    --
├─Linear: 1-1                            [1, 10]                   20
├─Linear: 1-2                            [1, 15]                   165
├─Linear: 1-3                            [1, 1]                    16
Total params: 201
Trainable params: 201
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00